In [ ]:
import cv2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
data = pd.read_csv("/content/sample_data/mnist_train_small.csv", header=None)
data

In [ ]:
targets = data[0]
data.drop(0, axis=1, inplace=True)

In [ ]:
twos = targets[targets == 2].index

In [ ]:
twos = data.loc[twos]

In [ ]:
sample_image = data.loc[5].values.reshape(28,28)

In [ ]:
plt.imshow(sample_image)

In [ ]:
!unzip /content/test_data.zip -d data/

In [ ]:
import os 
test_images = []
path = "/content/data/test_data"
for file in os.listdir(path):
  test_images.append(cv2.imread(path + "/" + file))



In [ ]:
plt.imshow(test_images[0])

In [ ]:
def eig(S):
    #calculate the eigen values and eigen vectors
    eig_val, eig_vec = np.linalg.eigh(S)  
    
    #sorting them in decrasing order
    sorted_eig  = np.argsort(-eig_val)
    eig_val = eig_val[sorted_eig]
    eig_vec = eig_vec[:, sorted_eig]

    return eig_val, eig_vec
    

def Standardize(X):
    #calcualte the mean of each column mu   
    mu = np.mean(X, axis = 0) 
    
    #Substract the mean from X
    X = X - mu  
    
    #calcualte the standard deviation of each column std
    std = np.std(X, axis = 0)  
    
    #handleing zero standard deviation
    std_filled = std.copy()
    std_filled[std == 0] = 1.0
    
    #calculate standardized X called Xbar
    Xbar = (X-mu) / std_filled  
    
    return Xbar, mu, std

def projection_matrix(B):
    #calculate the projection matrix P
    P = B @ B.T 
    
    return P

def PCA_(X):
    #calculate the data covariance matrix S
    S = np.cov(X)  
    
    #now find eigenvalues and corresponding eigenvectors for S by implementing eig().
    eig_vals, eig_vecs = eig(S)
    idx = eig_vals.argsort()[::-1]
    eig_vecs = eig_vecs[:, idx]
    eig_vals = eig_vals[idx]
    
    return eig_vecs, eig_vals


In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

In [ ]:
twos = twos// 255

In [ ]:
pca = PCA()
pca.fit(twos)

In [ ]:
sample_image_t = pca.transform(sample_image.flatten().reshape(1,-1))

In [ ]:


def reconstruction(image, N):
  img, std, mu = Standardize(image)
  img = pca.transform(sample_image.flatten().reshape(1,-1))
  eigenvec = pca.components_
  U = eigenvec[:, range(N)]
  P = projection_matrix(U)
  
  return P

In [ ]:
sample_image =sample_image.reshape(28,28)

In [ ]:
sample_image, mu, std = Standardize(sample_image)

In [ ]:
plt.imshow(sample_image)

In [ ]:
def pca_detect_digit(image, mean_digit, eigenv, N=28):
  from numpy import linalg
  s_bar, std, mu = Standardize(image)
  projection = PCA(s_bar, N)
  reconst = sample_image @ projection
  error = mse(reconst, sample_image)
  h, w = mean_digit.shape[:-1]
  mean_digit_grey = cv2.cvtColor(mean_digit, cv2.COLOR_BGR2GRAY)
  subwin_size = 28
  result = []
  centers = []

  for y in range(subwin_size, h):
       for x in range(subwin_size, w):
           mean_digit_piece = mean_digit_grey[y-subwin_size:y, x-subwin_size:x]
           
           diff = mse(mean_digit_piece, reconst)
           result.append(diff)
           centers.append([x,y])
  best_match = np.argmin(result, axis=0)
  k = centers[best_match]
  k1, k2 = k[0]//2, k[1]//2
  n = 14
  x1, x2, y1, y2 = k1-n, k1+n, k2-n, k2+n
  # plt.imshow(image)
  final = cv2.rectangle(mean_digit.copy(), (x1,y1), (x2, y2),(255,255,255),2)

  return final,centers


In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(15,15))
axes = axes.flatten()
for i, image in enumerate(test_images):
  axes[i].imshow(image, cmap='gray');
  # axes[i].set_title(f"Label: {}")

In [ ]:
result_images = []
for img in test_images:
  eigenv, eigval = PCA_(sample_image)
  # print(img.shape)
  final, centers = pca_detect_digit(sample_image, img, eigenv=eigenv, N = 10)
  result_images.append(final)

In [ ]:
centers[31]

In [ ]:
60/28

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(15,15))
axes = axes.flatten()
for i, image in enumerate(result_images):
  axes[i].imshow(image, cmap='gray');
  # axes[i].set_title(f"Label: {}")

In [ ]:
sample_image2 = sample_image.copy()
def Standardize(X):
    #calcualte the mean of each column mu   
    mu = np.mean(X, axis = 0) 
    
    #Substract the mean from X
    X = X - mu  
    
    #calcualte the standard deviation of each column std
    std = np.std(X, axis = 0)  
    
    #handleing zero standard deviation
    std_filled = std.copy()
    std_filled[std == 0] = 1.0
    
    #calculate standardized X called Xbar
    Xbar = (X-mu) / std_filled  
    
    return Xbar, mu, std


def eig(S):
    #calculate the eigen values and eigen vectors
    eig_val, eig_vec = np.linalg.eigh(S)  
    
    #sorting them in decrasing order
    sorted_eig  = np.argsort(-eig_val)
    eig_val = eig_val[sorted_eig]
    eig_vec = eig_vec[:, sorted_eig]
    
    return (eig_val, eig_vec)




def projection_matrix(B):
    #calculate the projection matrix P
    P = B @ B.T 
    
    return P


def PCA(X, num_components):
    #calculate the data covariance matrix S
    S = np.cov(X)  
    
    #now find eigenvalues and corresponding eigenvectors for S by implementing eig().
    eig_vals, eig_vecs = eig(S)
    
    #select eigen vectors
    U = eig_vecs[:, range(num_components)]
    
    #reconstruct the images from the lowerdimensional representation
    #to do this, we first need to find the projection_matrix (which you implemented earlier)
    #which projects our input data onto the vector space spanned by the eigenvectors
    P = projection_matrix(U) # projection matrix
    
    return P


Xbar_train, mu_train, std_train = Standardize(sample_image2)

def mse(predict, actual):
    return np.square(predict - actual).sum(axis = 0).mean()





In [ ]:

for num_component in range(1, 10):
    projection = PCA(Xbar_train.T, num_component)
    reconst = sample_image @ projection
    error = mse(reconst, sample_image)
    plt.imshow(reconst)

In [ ]:
sample_image.shape